In [21]:
import os
import json

import io
import os.path
import re
import tarfile
import sys
import gensim
import pandas as pd
import numpy as np


In [ ]:
repos_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

In [22]:
gutenberg_repo_path = os.path.join(repos_path, 'gutenberg')
gutenberg_analysis_repo = os.path.join(repos_path, 'gutenberg-analysis')

src_dir = os.path.join(gutenberg_analysis_repo,'src')
sys.path.append(src_dir)
from data_io import get_book


gutenberg_src_dir = os.path.join(gutenberg_repo_path,'src')
sys.path.append(gutenberg_src_dir)

from metaquery import meta_query


In [23]:
text_fold=os.path.join(gutenberg_repo_path, 'data', 'text')

In [24]:
train_csv = os.path.join('sample_dataset', 'train.csv')
test_csv = os.path.join('sample_dataset', 'test.csv')

In [25]:
train_df = pd.read_csv(train_csv, index_col='Unnamed: 0')
train_df.head()

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects
34653,PG41228,"Guy Deverell, v. 1 of 2","Le Fanu, Joseph Sheridan",1814.0,1873.0,['en'],88,"{'England -- Fiction', 'Country homes -- Ficti..."
28253,PG35468,"The Tenants of Malory, Volume 2","Le Fanu, Joseph Sheridan",1814.0,1873.0,['en'],109,{'Mystery fiction'}
18241,PG26451,"A Stable for Nightmares; or, Weird Tales","Le Fanu, Joseph Sheridan",1814.0,1873.0,['en'],203,{'Fantasy fiction'}
30146,PG37172,"In a Glass Darkly, v. 1/3","Le Fanu, Joseph Sheridan",1814.0,1873.0,['en'],325,"{'Ireland -- Fiction', 'Paranormal fiction'}"
34654,PG41229,"Guy Deverell, v. 2 of 2","Le Fanu, Joseph Sheridan",1814.0,1873.0,['en'],154,"{'England -- Fiction', 'Country homes -- Ficti..."


In [26]:
len(train_df['author'].unique())

173

In [27]:
subj = train_df['subjects'].replace('set()',np.nan)
subj_docs = []
for h in subj:
    try:
        h = h.strip("{}")[1:-1]
    except AttributeError:
        subj_docs.append(h)
        continue
    h = h.replace(' -- ', '-')
    h = h.replace("', '","_")
    h = h.split('_')
    h = [item.replace(' ','').replace(',', ' ') for item in h]
    h = ' '.join(h)
    subj_docs.append(h)

In [8]:
train_df['subj_str']=subj_docs

In [9]:
#train_df['subject_str'] = train_df['subjects'].apply(lambda x: split_subjects(x))

In [28]:
train_df['text'] = train_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))

In [29]:
train_df.dropna(subset=['text'], inplace=True)

In [12]:
#subject_docs = train_df['subjects'].tolist()

In [30]:

docs = train_df['text'].tolist()
#docs = train_df['subject_str'].tolist()

In [ ]:
docs[0]

In [26]:
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [27]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [28]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [29]:
import pickle
outfile = '/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/LDA/models/bigram_text_doc.pkl'
with open(outfile, 'wb') as fp:
    pickle.dump(docs, fp)

In [15]:
import pickle
outfile = '/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/LDA/models/bigram_text_doc.pkl'
with open(outfile, 'rb') as fp:
    docs = pickle.load(fp)

In [15]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [16]:
dictionary_outfile = '/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/LDA/models/bigram_dictionary_filt.dict'

In [20]:
dictionary.save(dictionary_outfile)

In [31]:
from gensim.corpora import Dictionary
dictionary = Dictionary.load(dictionary_outfile)

In [32]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# Training



## Setup Logging

In [33]:
import logging
# The filename is the file that will be created with the log.
# If the file already exists, the log will continue rather than being overwritten.
logging.basicConfig(filename='model_callbacks.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.NOTSET)
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [34]:
num_topics = 163

In [34]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50
chunksize = 2000
passes = 5
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [30]:
model_outfile = '/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/LDA/author_model_tut.mdl'
model.save(model_outfile)

In [20]:
from gensim.models import LdaModel
model_outfile = '/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/LDA/author_model_tut.mdl'
model  = LdaModel.load(model_outfile)

In [37]:
# Perplexity, lower is better
perplexity = model.log_perplexity(corpus)

In [38]:
perplexity

-9.287756556704219

In [35]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.6855.
[([(0.018918974, 'thou'),
   (0.01168854, 'thy'),
   (0.010225058, 'thee'),
   (0.0034767783, 'hath'),
   (0.0025580782, 'hast'),
   (0.0024869516, 'nay'),
   (0.0024142426, 'ti'),
   (0.002219124, 'er'),
   (0.002013665, 'unto'),
   (0.0019707917, 'knight'),
   (0.0018850135, 'thou_art'),
   (0.0017646217, 'honour'),
   (0.0017217726, 'thine'),
   (0.0016616783, 'thou_hast'),
   (0.0016108787, 'ere'),
   (0.0015733413, '_the'),
   (0.0014187479, 'doth'),
   (0.0013448676, 'saint'),
   (0.0013294864, 'poet'),
   (0.0013072871, 'behold')],
  -0.5905292423674808),
 ([(0.0038059023, 'mary'),
   (0.0032044426, 'james'),
   (0.0026355207, 'william'),
   (0.0022648901, 'inn'),
   (0.002213515, 'earl'),
   (0.0020430333, 'thomas'),
   (0.0018626512, 'duke'),
   (0.0018369796, 'holmes'),
   (0.0016754186, 'henry'),
   (0.0016351321, 'lane'),
   (0.0015095174, 'squire'),
   (0.0014967059, 'charles'),
   (0.0014798684, '_the'),
   (0.0014069509, 'mayor'),
   (0.

In [42]:
from gensim.models import CoherenceModel
coherencemodel = CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_val = coherencemodel.get_coherence()

/home/dean/anaconda3/envs/school-env/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: Th

In [36]:
pprint(model.print_topics())

[(36,
  '0.029*"fertilised" + 0.021*"seed" + 0.020*"styled" + 0.018*"pollen" + '
  '0.018*"nick" + 0.012*"capsule" + 0.010*"specie" + 0.010*"hallam" + '
  '0.009*"bayle" + 0.007*"stamen"'),
 (34,
  '0.009*"robert" + 0.007*"wi" + 0.006*"th" + 0.005*"hae" + 0.005*"cosmo" + '
  '0.004*"joan" + 0.003*"wad" + 0.003*"na" + 0.003*"wayne" + 0.003*"ken"'),
 (25,
  '0.017*"dan" + 0.014*"martin" + 0.014*"cub" + 0.012*"brad" + 0.009*"jonas" + '
  '0.008*"hatfield" + 0.008*"dick" + 0.005*"ira" + 0.005*"specie" + '
  '0.005*"prescott"'),
 (39,
  '0.008*"gland" + 0.008*"tentacle" + 0.006*"edith" + 0.005*"teacher" + '
  '0.005*"solution" + 0.004*"prevost" + 0.004*"dr" + 0.004*"walter" + '
  '0.003*"cell" + 0.003*"acid"'),
 (4,
  '0.010*"mary" + 0.009*"lucien" + 0.009*"annie" + 0.007*"levin" + 0.006*"jim" '
  '+ 0.004*"anna" + 0.003*"jerry" + 0.003*"kitty" + 0.003*"carve" + '
  '0.003*"aunt"'),
 (27,
  '0.015*"peter" + 0.012*"janet" + 0.007*"aunt" + 0.005*"bob" + 0.005*"johnny" '
  '+ 0.005*"rabbit" + 

# Plotting

In [39]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
vis

/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=64429) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/dean/anaconda3

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
43    -0.049844 -0.032889       1        1  7.370242
44    -0.006089  0.060556       2        1  6.387680
18     0.019312 -0.030261       3        1  5.180041
26    -0.025861 -0.035829       4        1  4.522341
49     0.005483 -0.002517       5        1  4.415305
19    -0.062935 -0.019229       6        1  3.479491
30     0.057795  0.011152       7        1  3.428973
13    -0.019433  0.046215       8        1  3.061794
32     0.007683 -0.012009       9        1  2.963661
7     -0.064328 -0.031457      10        1  2.959540
46     0.347649 -0.119166      11        1  2.484166
12    -0.044984 -0.031156      12        1  2.359725
20     0.077478 -0.007176      13        1  2.215457
48    -0.059108 -0.028419      14        1  2.038418
14    -0.068045 -0.043321      15        1  2.023666
23    -0.030171 -0.038766      16        1  2.014935
33     0.024644 -0.048961      17        1  1.930579
38    -0.040464 -0.025004      18        1  1.843410
21    -0.048647 -0.040106      19        1  1.745826
29    -0.045747 -0.036903      20        1  1.666715
45    -0.032402 -0.045511      21        1  1.653157
15    -0.049637  0.004605      22        1  1.602807
40    -0.030481 -0.007048      23        1  1.581341
9     -0.027584 -0.042247      24        1  1.563472
1      0.055031  0.173041      25        1  1.555590
47     0.258973 -0.092236      26        1  1.552178
8      0.043343  0.027835      27        1  1.492939
31    -0.065095 -0.032883      28        1  1.490344
42    -0.043106 -0.005316      29        1  1.415874
27    -0.057895 -0.042302      30        1  1.383359
2      0.076572  0.199080      31        1  1.373894
35     0.012144 -0.013943      32        1  1.353863
16     0.039211  0.119407      33        1  1.278409
3     -0.018714  0.004953      34        1  1.182250
11    -0.015132 -0.012042      35        1  1.162589
4     -0.046691 -0.035839      36        1  1.157399
34    -0.032738 -0.034218      37        1  1.151983
6     -0.044783 -0.031686      38        1  1.147021
37    -0.019659  0.004475      39        1  1.107773
24     0.011788  0.043446      40        1  1.096918
28    -0.032565 -0.070476      41        1  1.058116
10     0.041389 -0.054768      42        1  1.053134
5     -0.043683  0.002037      43        1  1.028703
0     -0.033896  0.003344      44        1  1.015184
41     0.011855 -0.023559      45        1  0.983954
22     0.009900 -0.051546      46        1  0.883978
17     0.034202  0.166559      47        1  0.803107
39    -0.006297  0.069454      48        1  0.706900
25     0.018530  0.113914      49        1  0.595255
36     0.013030  0.128717      50        1  0.476543, topic_info=         Term          Freq         Total Category  logprob  loglift
41779     inc  19169.000000  19169.000000  Default  30.0000  30.0000
3894      tom  33721.000000  33721.000000  Default  29.0000  29.0000
10218    thou  26016.000000  26016.000000  Default  28.0000  28.0000
41826    matt  12013.000000  12013.000000  Default  27.0000  27.0000
1165     dick  22716.000000  22716.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
5874    julia    363.865971   3989.905645  Topic50  -5.4927   2.9516
8769    union    405.216316   5879.964803  Topic50  -5.3851   2.6715
34785  gordon    321.092406   4460.518901  Topic50  -5.6178   2.7151
14978     mid    278.512045   2089.823452  Topic50  -5.7601   3.3310
13763  whilst    276.482666   8270.071020  Topic50  -5.7674   1.9481

[3093 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
34384      1  0.002074     1
34384      2  0.008296     1
34384      3  0.087111     1
34384      4  0.006222     1
34384      5  0.051852     1
...      ...       ...   ...
22664     33  0.004985  æsop
22664     34  0.712886  æsop
22664     41  0.004985  æsop
22664     45  0.